In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow as tf
from PIL import Image

# Load and preprocess the image
def load_and_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.vgg19.preprocess_input(img_array)
    return img_array

# Function to generate Grad-CAM heatmap
def generate_gradcam_heatmap(model, image_array, layer_name, class_index):
    preds = model.predict(image_array)
    top_class = preds[0].argmax() if class_index is None else class_index

    grad_model = tf.keras.models.Model([model.input], [model.get_layer(layer_name).output, model.output])

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(image_array)
        class_output = preds[0][top_class]

    grads = tape.gradient(class_output, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(1, 2))
    pooled_grads = tf.squeeze(pooled_grads)

    conv_layer_output_value = last_conv_layer_output[0].numpy()
    pooled_grads = tf.reshape(pooled_grads, (1, 1, 512))

    conv_layer_output_value *= pooled_grads.numpy()

    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) if np.max(heatmap) > 0 else 1

    return heatmap

# Function to generate Grad-CAM++ heatmap
def generate_gradcam_plus_plus(model, image_array, layer_name, class_index):
    preds = model.predict(image_array)
    top_class = preds[0].argmax() if class_index is None else class_index

    grad_model = tf.keras.models.Model([model.input], [model.get_layer(layer_name).output, model.output])

    with tf.GradientTape(persistent=True) as tape:
        last_conv_layer_output, preds = grad_model(image_array)
        class_output = preds[0][top_class]

    grads = tape.gradient(class_output, last_conv_layer_output)
    if grads is None:
        raise ValueError("Gradients for class output could not be computed.")

    second_grads = tape.gradient(grads, last_conv_layer_output)
    if second_grads is None:
        raise ValueError("Second gradients could not be computed.")

    pooled_grads = tf.reduce_mean(grads, axis=(1, 2))
    pooled_second_grads = tf.reduce_mean(second_grads, axis=(1, 2))

    conv_layer_output_value = last_conv_layer_output[0].numpy()
    
    weights = tf.nn.relu(pooled_grads) / (tf.nn.relu(pooled_grads) + tf.nn.relu(pooled_second_grads) + tf.keras.backend.epsilon())

    heatmap = np.zeros((conv_layer_output_value.shape[0], conv_layer_output_value.shape[1]))

    for i in range(conv_layer_output_value.shape[-1]):
        heatmap += weights[i] * conv_layer_output_value[:, :, i]

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) if np.max(heatmap) > 0 else 1

    return heatmap

# Function to generate Score-CAM heatmap
def generate_scorecam(model, image_array, layer_name, class_index):
    preds = model.predict(image_array)
    top_class = preds[0].argmax() if class_index is None else class_index

    grad_model = tf.keras.models.Model([model.input], [model.get_layer(layer_name).output, model.output])

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(image_array)
        class_output = preds[0][top_class]

    grads = tape.gradient(class_output, last_conv_layer_output)
    if grads is None:
        raise ValueError("Gradients for class output could not be computed.")

    conv_layer_output_value = last_conv_layer_output[0].numpy()

    # Calculate weights for Score-CAM
    weights = tf.nn.relu(grads[0])
    weights /= tf.reduce_sum(weights) + tf.keras.backend.epsilon()  # Normalize weights

    weights = tf.reshape(weights, (1, 1, -1))

    heatmap = np.zeros((conv_layer_output_value.shape[0], conv_layer_output_value.shape[1]))

    for i in range(conv_layer_output_value.shape[-1]):
        heatmap += weights[0, 0, i] * conv_layer_output_value[:, :, i]

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) if np.max(heatmap) > 0 else 1

    return heatmap

# Function to overlay heatmap on original image with red and orange colors
def overlay_heatmap(original, heatmap, alpha=0.5):
    heatmap_resized = np.uint8(255 * heatmap)
    heatmap_resized = np.squeeze(heatmap_resized)

    # Create a custom colormap
    cmap = plt.get_cmap('hot')  # Using the 'hot' colormap which is red to yellow
    heatmap_colored = cmap(heatmap_resized / 255.0)[:, :, :3]  # Get the RGB values
    heatmap_colored = np.array(heatmap_colored * 255, dtype=np.uint8)

    heatmap_colored = np.array(Image.fromarray(heatmap_colored).resize((original.shape[1], original.shape[0])))
    return np.clip(original + heatmap_colored * alpha, 0, 1)

# Load the VGG19 model
model = VGG19(weights='imagenet')

# Main code
image_path = '/kaggle/input/breast-xai-dataset/Breast_XAI/Benign_1.jpg'  # Update with your image path
image_array = load_and_preprocess_image(image_path)

# Generate heatmaps
layer_name = 'block5_conv4'
heatmap_gradcam = generate_gradcam_heatmap(model, image_array, layer_name, None)

try:
    heatmap_gradcam_plus_plus = generate_gradcam_plus_plus(model, image_array, layer_name, None)
except ValueError as e:
    print(e)
    heatmap_gradcam_plus_plus = np.zeros((14, 14))  # Placeholder

heatmap_scorecam = generate_scorecam(model, image_array, layer_name, None)

# Load original image for display
original_image = img_to_array(load_img(image_path, target_size=(224, 224))) / 255.0

# Overlay heatmaps on the original image
blended_image_gradcam = overlay_heatmap(original_image, heatmap_gradcam)
blended_image_gradcam_plus_plus = overlay_heatmap(original_image, heatmap_gradcam_plus_plus)
blended_image_scorecam = overlay_heatmap(original_image, heatmap_scorecam)

# Plot original image and blended images
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 1)
plt.imshow(original_image)
plt.title('Original Image')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.imshow(blended_image_gradcam)
plt.title('Grad-CAM Heatmap Overlay')
plt.axis('off')

plt.subplot(2, 2, 3)
plt.imshow(blended_image_gradcam_plus_plus)
plt.title('Grad-CAM++ Heatmap Overlay')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(blended_image_scorecam)
plt.title('Score-CAM Heatmap Overlay')
plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import os

# Function to save image
def save_image(image, path):
    img = Image.fromarray((image * 255).astype(np.uint8))
    img.save(path)

# Create a folder to save the images
output_folder = 'heatmap_outputs'
os.makedirs(output_folder, exist_ok=True)

# Paths for saving images
gradcam_path = os.path.join(output_folder, 'gradcam_overlay.png')
gradcam_plus_plus_path = os.path.join(output_folder, 'gradcam_plus_plus_overlay.png')
scorecam_path = os.path.join(output_folder, 'scorecam_overlay.png')

# Save the blended images
save_image(blended_image_gradcam, gradcam_path)
save_image(blended_image_gradcam_plus_plus, gradcam_plus_plus_path)
save_image(blended_image_scorecam, scorecam_path)

# Optionally, print the paths to verify
print(f"Images saved to: {output_folder}")
print(f"Grad-CAM overlay saved to: {gradcam_path}")
print(f"Grad-CAM++ overlay saved to: {gradcam_plus_plus_path}")
print(f"Score-CAM overlay saved to: {scorecam_path}")

# Plot original image and blended images (remains unchanged)
plt.figure(figsize=(15, 10))
plt.subplot(2, 2, 1)
plt.imshow(original_image)
plt.title('Original Image')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.imshow(blended_image_gradcam)
plt.title('Grad-CAM Heatmap Overlay')
plt.axis('off')

plt.subplot(2, 2, 3)
plt.imshow(blended_image_gradcam_plus_plus)
plt.title('Grad-CAM++ Heatmap Overlay')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(blended_image_scorecam)
plt.title('Score-CAM Heatmap Overlay')
plt.axis('off')

plt.tight_layout()
plt.show()
